In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yahoo-finance-stock-market-data/stock_market_data.csv


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317146 sha256=e04376faedb72c40649103a603678a7148b6a86ea37206c85ca3fa3f51743c71
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when
from pyspark.sql.window import Window
import datetime
import warnings
warnings.simplefilter("ignore")

# from datetime import datetime
sc = SparkContext('local')


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/20 02:06:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark = SparkSession(sc)
df=spark.read.format("csv").option("header","true").load('/kaggle/input/yahoo-finance-stock-market-data/stock_market_data.csv')
df.show()

+----------+---------+---------+---------+---------+
|      Date|     AMZN|     GOOG|     QCOM|     AMAT|
+----------+---------+---------+---------+---------+
|02/01/2020|94.900497|  68.3685|88.690002|62.200001|
|03/01/2020|93.748497|68.032997|87.019997|61.209999|
|06/01/2020|95.143997|69.710503|86.510002|59.889999|
|07/01/2020|95.343002|   69.667|88.970001|61.619999|
|08/01/2020|94.598503|70.216003|88.709999|61.580002|
|09/01/2020|95.052498|70.991501|89.910004|61.970001|
|10/01/2020|94.157997|71.486504|90.260002|61.470001|
|13/01/2020|94.565002|71.961502|90.970001|61.810001|
|14/01/2020|   93.472|71.543999|90.559998|62.349998|
|15/01/2020|93.100998|71.959999|89.669998|    61.93|
|16/01/2020|93.897003|72.584999|91.790001|63.209999|
|17/01/2020|   93.236|74.019501|95.910004|62.849998|
|21/01/2020|94.599998|74.220001|94.540001|63.200001|
|22/01/2020|94.373001|74.297501|92.970001|63.889999|
|23/01/2020|94.228996|74.332497|       92|63.889999|
|24/01/2020|93.082001|73.335503|89.650002|    

In [5]:
warnings.simplefilter("ignore")
df = df.withColumn("Date", F.to_date(F.col("Date"), 'dd/MM/yyyy'))
min_date = df.select(F.min("Date")).first()[0]
max_date = df.select(F.max("Date")).first()[0]
date_range = [min_date + datetime.timedelta(days=x) for x in range((max_date - min_date).days + 1)]
all_dates_df = spark.createDataFrame([(date,) for date in date_range], ["Date"])
full_df = all_dates_df.join(df, on="Date", how="left_outer").orderBy("Date")
window = (Window.orderBy('Date').rowsBetween(Window.unboundedPreceding, Window.currentRow))
full_df = (full_df.withColumn('AMZN', F.last('AMZN', ignorenulls=True).over(window)))
full_df = (full_df.withColumn('GOOG', F.last('AMZN', ignorenulls=True).over(window)))
full_df = (full_df.withColumn('QCOM', F.last('AMZN', ignorenulls=True).over(window)))
full_df = (full_df.withColumn('AMAT', F.last('AMZN', ignorenulls=True).over(window)))
    
full_df.show()


23/06/20 02:06:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/20 02:06:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/20 02:06:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/20 02:06:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/20 02:06:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/20 02:06:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/20 0

+----------+---------+---------+---------+---------+
|      Date|     AMZN|     GOOG|     QCOM|     AMAT|
+----------+---------+---------+---------+---------+
|2020-01-02|94.900497|94.900497|94.900497|94.900497|
|2020-01-03|93.748497|93.748497|93.748497|93.748497|
|2020-01-04|93.748497|93.748497|93.748497|93.748497|
|2020-01-05|93.748497|93.748497|93.748497|93.748497|
|2020-01-06|95.143997|95.143997|95.143997|95.143997|
|2020-01-07|95.343002|95.343002|95.343002|95.343002|
|2020-01-08|94.598503|94.598503|94.598503|94.598503|
|2020-01-09|95.052498|95.052498|95.052498|95.052498|
|2020-01-10|94.157997|94.157997|94.157997|94.157997|
|2020-01-11|94.157997|94.157997|94.157997|94.157997|
|2020-01-12|94.157997|94.157997|94.157997|94.157997|
|2020-01-13|94.565002|94.565002|94.565002|94.565002|
|2020-01-14|   93.472|   93.472|   93.472|   93.472|
|2020-01-15|93.100998|93.100998|93.100998|93.100998|
|2020-01-16|93.897003|93.897003|93.897003|93.897003|
|2020-01-17|   93.236|   93.236|   93.236|   9

In [6]:
full_df.toPandas().to_csv('pre_processed_data.csv',header=True)

23/06/20 02:06:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/20 02:06:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/20 02:06:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/20 02:06:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/20 02:06:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/20 02:06:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/20 0

In [7]:
stock_data = pd.read_csv('/kaggle/working/pre_processed_data.csv')
stock_data = stock_data.drop(columns=['Unnamed: 0'])
stock_data.head()

,Date,AMZN,GOOG,QCOM,AMAT
0,2020-01-02,94.900497,94.900497,94.900497,94.900497
1,2020-01-03,93.748497,93.748497,93.748497,93.748497
2,2020-01-04,93.748497,93.748497,93.748497,93.748497
3,2020-01-05,93.748497,93.748497,93.748497,93.748497
4,2020-01-06,95.143997,95.143997,95.143997,95.143997


THE SHARE PRICES ARE IN POUNDS AND NOT IN DOLLARS

# Autoregression (AR) model